In [0]:
from pyspark.sql.functions import trim, col
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType
from pyspark.sql.functions import col, date_format 

In [0]:
dbutils.secrets.listScopes()

In [0]:
dbutils.fs.mount(
    source='wasbs://spotify@moviesdatajcs.blob.core.windows.net',
    mount_point='/mnt/spotify_data',
    extra_configs = {'fs.azure.account.key.moviesdatajcs.blob.core.windows.net':
                     dbutils.secrets.get('movieDataSecretScope', 'moviesStorageAccountKey')} 
)

In [0]:
df = spark.read.option("header", True).csv("dbfs:/mnt/spotify_data/raw-data/datasetSpotify.csv")
df.display()

In [0]:
#drop any rows with null values
df_clean = df.dropna()

#trim whitespace from all string columns
for column in df_clean.columns:
    df_clean = df_clean.withColumn(column, trim(col(column)))

df_clean.display()

In [0]:
df_clean.printSchema()

In [0]:
df_clean = df_clean.withColumn("song_popularity", col("song_popularity").cast(IntegerType()))
df_clean = df_clean.withColumn("artist_popularity", col("artist_popularity").cast(IntegerType()))
df_clean.printSchema()

In [0]:
spotify_tracks = df_clean

spotify_tracks_byPopularArtist = spotify_tracks.orderBy("artist_popularity", ascending = False).select("ID", "song", "artist", "album", "genre").limit(50).show()

In [0]:
spotify_tracks_pop = spotify_tracks.filter(spotify_tracks["genre"] == "pop").select("ID", "song", "artist", "album", "genre").show()

In [0]:
spotify_tracks_top25Artists = spotify_tracks.orderBy("artist").select("ID", "artist", "artist_popularity").limit(25).show()

In [0]:
display(spotify_tracks_byPopularArtist)